<a href="https://colab.research.google.com/github/Jorge2018/DataScience2022-2/blob/main/Script_actividadPipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
########### Codigo lee archivo excel desde github
import pandas as pd
import numpy as np
from pandas import DataFrame
import re
import requests
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')


url = "https://github.com/Jorge2018/RepositorioArchivos/blob/main/Cereal%20with%20missing%20values.xlsx?raw=true"

response = requests.get(url)

dest = 'local-file.xlsx'

with open(dest, 'wb') as file:
    file.write(response.content)

df = pd.read_excel(dest)

df.head()



,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2.0,2.0,180.0,1.5,10.5,10.0,70.0,25.0,1.0,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3.0,2.0,NaN,2.0,18.0,NaN,100.0,25.0,3.0,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6.0,2.0,290.0,2.0,17.0,1.0,105.0,25.0,1.0,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1.0,3.0,210.0,0.0,13.0,9.0,45.0,25.0,2.0,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3.0,2.0,140.0,2.0,13.0,7.0,105.0,25.0,3.0,1.00,0.50,40.400208


**Reduccion de cols, solo considera aquellas que menciona el enunciado**

In [91]:
df=df[['Manufacturer','type','calories per serving','grams of fat','grams of sugars','Weight in ounces per one serving']]
df

,Manufacturer,type,calories per serving,grams of fat,grams of sugars,Weight in ounces per one serving
0,General Mills,Cold,110.0,2.0,10.0,1.00
1,General Mills,Cold,130.0,2.0,NaN,1.33
2,General Mills,Cold,NaN,2.0,1.0,1.00
3,General Mills,Cold,120.0,3.0,9.0,1.00
4,General Mills,Cold,110.0,2.0,7.0,1.00
...,...,...,...,...,...,...
72,Ralston Purina,Cold,110.0,NaN,2.0,1.00
73,Ralston Purina,Cold,100.0,1.0,3.0,1.00
74,American Home Food Products,Hot,100.0,1.0,NaN,1.00
75,Nabisco,Hot,100.0,0.0,0.0,1.00


**Inspeccion de datos**

In [92]:
#inspect the data
print(df.info(), '\n')
print('________________________________________________________')
print('')
print('los datos null son')
print(df.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Manufacturer                      77 non-null     object 
 1   type                              68 non-null     object 
 2   calories per serving              70 non-null     float64
 3   grams of fat                      69 non-null     float64
 4   grams of sugars                   68 non-null     float64
 5   Weight in ounces per one serving  77 non-null     float64
dtypes: float64(4), object(2)
memory usage: 3.7+ KB
None 

________________________________________________________

los datos null son
Manufacturer                        0
type                                9
calories per serving                7
grams of fat                        8
grams of sugars                     9
Weight in ounces per one serving    0
dtype: int64


**Separa valores en X, y**

In [93]:
# Dividan
X = df.drop('calories per serving', axis=1)
y = df['calories per serving']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)



**Instanciar selectores**

In [94]:
# Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')



**Instanciar transformadores SimpleImputer, StandardScaler y OneHotEncoder**

In [95]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)



**Instanciar el pipeline**

In [96]:
# Numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

**Instanciar el pipeline**

In [97]:
# Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

**Instanciar ColumnTransformer**

In [98]:
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor



ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7faf8c3a5ad0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7faf8c3a5ed0>)])

**Transformador de datos**

In [99]:
# fit on train
preprocessor.fit(X_train)
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)




**Inspeccion de resultados**

In [100]:
# Comprueben los valores faltantes y que los datos se escalen y tengan una codificación one-hot
print(np.isnan(X_train_processed).sum().sum(), 'valores perdidos en la data de entrenamiento')
print(np.isnan(X_test_processed).sum().sum(), 'valores perdidos en la data de prueba')
print('\n')
print('Todos los datos procesados del X_train_processed son del tipo:', X_train_processed.dtype)
print('Todos los datos procesados del X_test_processed son del tipo', X_test_processed.dtype)
print('\n')
print('shape de los datos de entrenamiento son', X_train_processed.shape)
print('\n')
X_train_processed




0 valores perdidos en la data de entrenamiento
0 valores perdidos en la data de prueba


Todos los datos procesados del X_train_processed son del tipo: float64
Todos los datos procesados del X_test_processed son del tipo float64


shape de los datos de entrenamiento son (57, 11)




array([[-9.74679434e-01,  9.94481647e-01, -1.32764897e-01,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.22191915e+00,  2.03880702e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [-9.74679434e-01, -8.25018407e-01, -1.32764897e-01,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.67679417e+00,  3.15749558e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.42705887e-01, -1.32764897e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  